In [1]:
#autopilot based on minimap path
import cv2 as cv
import numpy as np
from PIL import ImageGrab #screen rec
from IPython.display import clear_output
import keyboard


def thresholding(miniMap):
    #define range of purple color in HSV
    lower = np.array([50,160,240])
    upper = np.array([200,180,250])
    
    #threshold the HSV image to get only blue colors
    mask = cv.inRange(hsv, lower, upper)

    res = cv.bitwise_and(miniMap,miniMap, mask = mask)
    
    return mask, res

#----------------------------------------------------------------------------

def warpImg(img):
    #220x367
    #upper points
    cv.circle(miniMap, (141,110), 3, (0,0,255))
    cv.circle(miniMap, (222,110), 3, (0,255,0))
    #lower points
    cv.circle(miniMap, (135,200), 3, (0,0,255))
    cv.circle(miniMap, (232,200), 3, (0,255,0))
    
    pts1 = np.float32([[141,110],[222,110],[135,200],[232,200]])
    pts2 = np.float32([[0,0],[500,0],[0,500],[500,500]])

    M = cv.getPerspectiveTransform(pts1,pts2)

    dst = cv.warpPerspective(img,M,(500,500))

    return dst

#-------------------------------------------------------------------------------

def getHist(img, minPer = 0.1, region = 1):
    if region == 1:
        histVal = np.sum(img, axis = 0)
    else:
        histVal = np.sum(img[img.shape[0]//region:,:], axis = 0)
        
    maxVal = np.max(histVal)
    minVal = minPer * maxVal
    indexArray = np.where(histVal >= minVal)
    basePoint = np.average(indexArray)
    imgHist = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    
    for x,intensity in enumerate(histVal):
        cv.line(imgHist,(x, img.shape[0]), (x,img.shape[0]-intensity//255//region) , (255,0,100), 1)
        cv.circle(imgHist, (int(basePoint), img.shape[0] - 30), 25, (0,80,220), cv.FILLED)

    return basePoint, imgHist

#---------------------------------------------------------------------------

curveList = []
avgVal = 10

while True:
    clear_output()
    
    miniMap = np.array(ImageGrab.grab(bbox=(35,1170,402,1390))) #res = 220x367w
    miniMap = cv.cvtColor(miniMap, cv.COLOR_RGB2BGR)
    hsv = cv.cvtColor(miniMap, cv.COLOR_BGR2HSV)
    
    mask, res = thresholding(miniMap)[0], thresholding(miniMap)[1]
    
    dst = warpImg(mask)
    
    height, width = dst.shape
    
    left = dst[0:height, 0:int(width/2)-45]
    right = dst[0:height, int(width/2)+45:width]
    
    curveAveragePoint, imgHist = getHist(dst, minPer=0.9)
    midPoint, imgHist = getHist(dst, minPer=0.5, region=10)
    curveRaw = curveAveragePoint - midPoint
    
    curveList.append(curveRaw)
    if len(curveList) > avgVal:
        curveList.pop(0)
    curve = int(sum(curveList)) /len(curveList)
    
    #cv.imshow('img', miniMap)
    #cv.imshow('mask', mask)
    #cv.imshow('res', res)
    #cv.imshow('dst', dst)
    #cv.imshow('right', right)
    #cv.imshow('left', left)
    cv.imshow('imgHist', imgHist)
    
    
    k = cv.waitKey(20)
    if k == 113:
        break
        
cv.destroyAllWindows()